In [2]:
import tensorflow as tf
from tensorflow.keras import layers
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import glob
import keras

In [2]:
data = np.load("./data/data_preprocessed_1.npz") #loading data

In [106]:
X = data["arr_0"]
y = data["arr_1"]

In [4]:
X.shape

(6058, 224, 224, 3)

In [5]:
y.shape

(6058,)

In [6]:
from sklearn.model_selection import train_test_split
X_train , X_test , y_train , y_test = train_test_split(X , y , test_size=0.2, random_state=42)


In [7]:
X_train.shape , X_test.shape ,y_train.shape , y_test.shape

((4846, 224, 224, 3), (1212, 224, 224, 3), (4846,), (1212,))

In [95]:
model = tf.keras.applications.MobileNetV2()  

In [11]:
model.summary()

Model: "mobilenetv2_1.00_224"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 Conv1 (Conv2D)                 (None, 112, 112, 32  864         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 bn_Conv1 (BatchNormalization)  (None, 112, 112, 32  128         ['Conv1[0][0]']                  
                                )                                              

 block_3_expand_relu (ReLU)     (None, 56, 56, 144)  0           ['block_3_expand_BN[0][0]']      
                                                                                                  
 block_3_pad (ZeroPadding2D)    (None, 57, 57, 144)  0           ['block_3_expand_relu[0][0]']    
                                                                                                  
 block_3_depthwise (DepthwiseCo  (None, 28, 28, 144)  1296       ['block_3_pad[0][0]']            
 nv2D)                                                                                            
                                                                                                  
 block_3_depthwise_BN (BatchNor  (None, 28, 28, 144)  576        ['block_3_depthwise[0][0]']      
 malization)                                                                                      
                                                                                                  
 block_3_d

 lization)                                                                                        
                                                                                                  
 block_7_expand (Conv2D)        (None, 14, 14, 384)  24576       ['block_6_project_BN[0][0]']     
                                                                                                  
 block_7_expand_BN (BatchNormal  (None, 14, 14, 384)  1536       ['block_7_expand[0][0]']         
 ization)                                                                                         
                                                                                                  
 block_7_expand_relu (ReLU)     (None, 14, 14, 384)  0           ['block_7_expand_BN[0][0]']      
                                                                                                  
 block_7_depthwise (DepthwiseCo  (None, 14, 14, 384)  3456       ['block_7_expand_relu[0][0]']    
 nv2D)    

                                                                                                  
 block_10_depthwise_relu (ReLU)  (None, 14, 14, 384)  0          ['block_10_depthwise_BN[0][0]']  
                                                                                                  
 block_10_project (Conv2D)      (None, 14, 14, 96)   36864       ['block_10_depthwise_relu[0][0]']
                                                                                                  
 block_10_project_BN (BatchNorm  (None, 14, 14, 96)  384         ['block_10_project[0][0]']       
 alization)                                                                                       
                                                                                                  
 block_11_expand (Conv2D)       (None, 14, 14, 576)  55296       ['block_10_project_BN[0][0]']    
                                                                                                  
 block_11_

 block_14_depthwise (DepthwiseC  (None, 7, 7, 960)   8640        ['block_14_expand_relu[0][0]']   
 onv2D)                                                                                           
                                                                                                  
 block_14_depthwise_BN (BatchNo  (None, 7, 7, 960)   3840        ['block_14_depthwise[0][0]']     
 rmalization)                                                                                     
                                                                                                  
 block_14_depthwise_relu (ReLU)  (None, 7, 7, 960)   0           ['block_14_depthwise_BN[0][0]']  
                                                                                                  
 block_14_project (Conv2D)      (None, 7, 7, 160)    153600      ['block_14_depthwise_relu[0][0]']
                                                                                                  
 block_14_

# Transferleaning

In [96]:
model_input = model.layers[0].input

In [97]:
model_input

<KerasTensor: shape=(None, 224, 224, 3) dtype=float32 (created by layer 'input_2')>

In [98]:
primary_output = model.layers[-2].output

In [99]:
primary_output 

<KerasTensor: shape=(None, 1280) dtype=float32 (created by layer 'global_average_pooling2d_1')>

In [100]:
final_output = layers.Dense(128 , activation = "relu")(primary_output)  # adding layers after global pooling layer
final_output = layers.Activation("relu")(final_output)

final_output = layers.Dense(64, activation = "relu")(final_output)
final_output = layers.Activation("relu")(final_output)

output = layers.Dense(1 , activation = "sigmoid")(final_output) #since there  are 2 classes


In [101]:
final_model = keras.Model(inputs = model_input , outputs = output )

In [102]:
final_model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 Conv1 (Conv2D)                 (None, 112, 112, 32  864         ['input_2[0][0]']                
                                )                                                                 
                                                                                                  
 bn_Conv1 (BatchNormalization)  (None, 112, 112, 32  128         ['Conv1[0][0]']                  
                                )                                                           

 block_3_expand_relu (ReLU)     (None, 56, 56, 144)  0           ['block_3_expand_BN[0][0]']      
                                                                                                  
 block_3_pad (ZeroPadding2D)    (None, 57, 57, 144)  0           ['block_3_expand_relu[0][0]']    
                                                                                                  
 block_3_depthwise (DepthwiseCo  (None, 28, 28, 144)  1296       ['block_3_pad[0][0]']            
 nv2D)                                                                                            
                                                                                                  
 block_3_depthwise_BN (BatchNor  (None, 28, 28, 144)  576        ['block_3_depthwise[0][0]']      
 malization)                                                                                      
                                                                                                  
 block_3_d

 lization)                                                                                        
                                                                                                  
 block_7_expand (Conv2D)        (None, 14, 14, 384)  24576       ['block_6_project_BN[0][0]']     
                                                                                                  
 block_7_expand_BN (BatchNormal  (None, 14, 14, 384)  1536       ['block_7_expand[0][0]']         
 ization)                                                                                         
                                                                                                  
 block_7_expand_relu (ReLU)     (None, 14, 14, 384)  0           ['block_7_expand_BN[0][0]']      
                                                                                                  
 block_7_depthwise (DepthwiseCo  (None, 14, 14, 384)  3456       ['block_7_expand_relu[0][0]']    
 nv2D)    

                                                                                                  
 block_10_depthwise_relu (ReLU)  (None, 14, 14, 384)  0          ['block_10_depthwise_BN[0][0]']  
                                                                                                  
 block_10_project (Conv2D)      (None, 14, 14, 96)   36864       ['block_10_depthwise_relu[0][0]']
                                                                                                  
 block_10_project_BN (BatchNorm  (None, 14, 14, 96)  384         ['block_10_project[0][0]']       
 alization)                                                                                       
                                                                                                  
 block_11_expand (Conv2D)       (None, 14, 14, 576)  55296       ['block_10_project_BN[0][0]']    
                                                                                                  
 block_11_

 block_14_depthwise (DepthwiseC  (None, 7, 7, 960)   8640        ['block_14_expand_relu[0][0]']   
 onv2D)                                                                                           
                                                                                                  
 block_14_depthwise_BN (BatchNo  (None, 7, 7, 960)   3840        ['block_14_depthwise[0][0]']     
 rmalization)                                                                                     
                                                                                                  
 block_14_depthwise_relu (ReLU)  (None, 7, 7, 960)   0           ['block_14_depthwise_BN[0][0]']  
                                                                                                  
 block_14_project (Conv2D)      (None, 7, 7, 160)    153600      ['block_14_depthwise_relu[0][0]']
                                                                                                  
 block_14_

In [104]:
#compiling model
final_model.compile(loss = tf.keras.losses.binary_crossentropy, 
                   optimizer = tf.keras.optimizers.Adam(learning_rate = 0.001),
                   metrics = ["accuracy"])

In [107]:
final_model.fit(X , y, epochs = 15)

Epoch 1/15
190/190 [==============================] - 1582s 7s/step - loss: 0.3357 - accuracy: 0.8496
Epoch 2/15
190/190 [==============================] - 1234s 6s/step - loss: 0.2137 - accuracy: 0.9173
Epoch 3/15
190/190 [==============================] - 1085s 6s/step - loss: 0.1544 - accuracy: 0.9416
Epoch 4/15
190/190 [==============================] - 1086s 6s/step - loss: 0.1352 - accuracy: 0.9473
Epoch 5/15
190/190 [==============================] - 1096s 6s/step - loss: 0.1239 - accuracy: 0.9548
Epoch 6/15
190/190 [==============================] - 1081s 6s/step - loss: 0.0933 - accuracy: 0.9675
Epoch 7/15
190/190 [==============================] - 1088s 6s/step - loss: 0.0920 - accuracy: 0.9670
Epoch 8/15
190/190 [==============================] - 1178s 6s/step - loss: 0.0718 - accuracy: 0.9744
Epoch 9/15
190/190 [==============================] - 1312s 7s/step - loss: 0.0677 - accuracy: 0.9751
Epoch 10/15
190/190 [==============================] - 1325s 7s/step - loss: 0.061

In [109]:
final_model.save("./model/model_mobile_net_2.h5")

In [3]:
model = keras.models.load_model("./model/model_mobile_net_2.h5")

In [111]:
model.evaluate(X, y)

190/190 [==============================] - 282s 1s/step - loss: 0.2514 - accuracy: 0.9206


[0.2514302134513855, 0.9206008315086365]

In [122]:
test_image_fe = cv2.imread("./data/download.jpg")

In [30]:
test_img_m = cv2.imread("./data/male_000281.jpg")

In [31]:
test_img_m.shape

(500, 500, 3)

In [17]:
Haar = cv2.CascadeClassifier(r"C:\Users\nared\anaconda3\Lib\site-packages\cv2\data/haarcascade_frontalface_default.xml")

In [7]:
face = Haar.detectMultiScale(test_image , 1.5 , 5)
for x,y,h,w in face :
    roi = test_image[y:y+h , x:x+w]
# cv2.imwrite("./data/female_test_processed_1.png", roi)       

NameError: name 'test_image' is not defined

In [123]:
roi.shape

(122, 122, 3)

In [114]:
test_image.shape

(134, 186, 3)

In [124]:
test_image_fe = cv2.resize(roi , (224,224))

In [125]:
test_image_fe = test_image_fe/225.

In [126]:
test_image_fe = np.reshape(test_image_fe,[1,224,224,3])

In [128]:
model.predict(test_image_fe)

1/1 [==============================] - 0s 198ms/step


array([[0.9568553]], dtype=float32)

In [32]:
face = Haar.detectMultiScale(test_img_m , 1.5 , 5)
for x,y,h,w in face :
    roi = test_img_m[y:y+h , x:x+w]   

In [33]:
test_img_m.shape

(500, 500, 3)

In [35]:
test_image_m = cv2.resize(roi , (224,224))
test_image_m = test_image_m/225.
test_image_m = np.reshape(test_image_m,[1,224,224,3])

In [37]:
model.predict(test_image_m)

1/1 [==============================] - 0s 60ms/step


array([[1.2340153e-06]], dtype=float32)

## Labels = {"Female" : 1 , "Male" : 1}

In [39]:
1 - model.predict(test_image_m)[0][0]

1/1 [==============================] - 0s 54ms/step


0.999998765984742